In [1]:
import json
import requests
from pandas import json_normalize
import pandas as pd
!pip install dictor



# The Problem:
### The Foursquare Category Hierarchy has lists containing dictionary values containing lists and so on.

We'll need to download the hierarchy and create a recursive algorithm to "flatten" it all out

In [2]:
def getCategoryHierarchy():
    
    url =  'https://api.foursquare.com/v2/venues/categories'
    params = dict(
    client_id = 'FQATKQEFO0CWQSG0GGXEEHPOPQJSFZ4N5T1H2FBZMTKR4CU4',
    client_secret = '2Z5WI2MSN5A54B44OLHURZGWUYNEATSKU5QNGWLIOFEA05H2',
    v = '20180604',
    )

    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    
    return(data)


In [3]:
#categories = pd.DataFrame()
response = getCategoryHierarchy()

In [4]:
response

{'meta': {'code': 200, 'requestId': '6004993fc8ba5b3c6d78b2fb'},
 'response': {'categories': [{'id': '4d4b7104d754a06370d81259',
    'name': 'Arts & Entertainment',
    'pluralName': 'Arts & Entertainment',
    'shortName': 'Arts & Entertainment',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
     'suffix': '.png'},
    'categories': [{'id': '56aa371be4b08b9a8d5734db',
      'name': 'Amphitheater',
      'pluralName': 'Amphitheaters',
      'shortName': 'Amphitheater',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4fceea171983d5d06c3e9823',
      'name': 'Aquarium',
      'pluralName': 'Aquariums',
      'shortName': 'Aquarium',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
       'suffix': '.png'},
      'categories': []},
     {'id': '4bf58dd8d48988d1e1931735',
      'name': 'A

In [5]:
bigCategories = response['response']['categories']
bigCategories

[{'id': '4d4b7104d754a06370d81259',
  'name': 'Arts & Entertainment',
  'pluralName': 'Arts & Entertainment',
  'shortName': 'Arts & Entertainment',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
   'suffix': '.png'},
  'categories': [{'id': '56aa371be4b08b9a8d5734db',
    'name': 'Amphitheater',
    'pluralName': 'Amphitheaters',
    'shortName': 'Amphitheater',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_',
     'suffix': '.png'},
    'categories': []},
   {'id': '4fceea171983d5d06c3e9823',
    'name': 'Aquarium',
    'pluralName': 'Aquariums',
    'shortName': 'Aquarium',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_',
     'suffix': '.png'},
    'categories': []},
   {'id': '4bf58dd8d48988d1e1931735',
    'name': 'Arcade',
    'pluralName': 'Arcades',
    'shortName': 'Arcade',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_ent

In [6]:
#just for reference as I sought to troubleshoot below
cat_df = pd.DataFrame(bigCategories)
cat_df

,id,name,pluralName,shortName,icon,categories
0,4d4b7104d754a06370d81259,Arts & Entertainment,Arts & Entertainment,Arts & Entertainment,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '56aa371be4b08b9a8d5734db', 'name': 'A..."
1,4d4b7105d754a06372d81259,College & University,Colleges & Universities,College & Education,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C..."
2,4d4b7105d754a06373d81259,Event,Events,Event,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '52f2ab2ebcbc57f1066b8b3b', 'name': 'C..."
3,4d4b7105d754a06374d81259,Food,Food,Food,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '503288ae91d4c4b30a586d67', 'name': 'A..."
4,4d4b7105d754a06376d81259,Nightlife Spot,Nightlife Spots,Nightlife,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B..."
5,4d4b7105d754a06377d81259,Outdoors & Recreation,Outdoors & Recreation,Outdoors & Recreation,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A..."
6,4d4b7105d754a06375d81259,Professional & Other Places,Professional & Other Places,Professional,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4e52d2d203646f7c19daa8ae', 'name': 'A..."
7,4e67e38e036454776db1fb3a,Residence,Residences,Residence,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A..."
8,4d4b7105d754a06378d81259,Shop & Service,Shops & Services,Shops,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '52f2ab2ebcbc57f1066b8b56', 'name': 'A..."
9,4d4b7105d754a06379d81259,Travel & Transport,Travel & Transport,Travel,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A..."


In [7]:
# create recursive algorithm to find dictionaries nested within lists nested within dictionaries etc. 

smallCatList = []


def iterdict(d):
    if isinstance(d, list): 
        for v in d:
            print("\nList:\n")
            iterdict(v)
        
    elif isinstance(d, dict): 
        for k,v in d.items():
            if isinstance(v, dict):     
                iterdict(v)
                
                
            elif isinstance(v,list):
                for subvalue in v:
                    iterdict(subvalue)
                       
            else:            
                print (k,":",v)
                if k == 'name':
                    smallCatList.append(v)
    else:
        print("something went wrong")
        
    

iterdict(bigCategories)


List:

id : 4d4b7104d754a06370d81259
name : Arts & Entertainment
pluralName : Arts & Entertainment
shortName : Arts & Entertainment
prefix : https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_
suffix : .png
id : 56aa371be4b08b9a8d5734db
name : Amphitheater
pluralName : Amphitheaters
shortName : Amphitheater
prefix : https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_
suffix : .png
id : 4fceea171983d5d06c3e9823
name : Aquarium
pluralName : Aquariums
shortName : Aquarium
prefix : https://ss3.4sqi.net/img/categories_v2/arts_entertainment/aquarium_
suffix : .png
id : 4bf58dd8d48988d1e1931735
name : Arcade
pluralName : Arcades
shortName : Arcade
prefix : https://ss3.4sqi.net/img/categories_v2/arts_entertainment/arcade_
suffix : .png
id : 4bf58dd8d48988d1e2931735
name : Art Gallery
pluralName : Art Galleries
shortName : Art Gallery
prefix : https://ss3.4sqi.net/img/categories_v2/arts_entertainment/artgallery_
suffix : .png
id : 4bf58dd8d48988d1e4931735
name : B

shortName : Cambodian
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 4bf58dd8d48988d145941735
name : Chinese Restaurant
pluralName : Chinese Restaurants
shortName : Chinese
prefix : https://ss3.4sqi.net/img/categories_v2/food/asian_
suffix : .png
id : 52af3a5e3cf9994f4e043bea
name : Anhui Restaurant
pluralName : Anhui Restaurants
shortName : Anhui
prefix : https://ss3.4sqi.net/img/categories_v2/food/asian_
suffix : .png
id : 52af3a723cf9994f4e043bec
name : Beijing Restaurant
pluralName : Beijing Restaurants
shortName : Beijing
prefix : https://ss3.4sqi.net/img/categories_v2/food/asian_
suffix : .png
id : 52af3a7c3cf9994f4e043bed
name : Cantonese Restaurant
pluralName : Cantonese Restaurants
shortName : Cantonese
prefix : https://ss3.4sqi.net/img/categories_v2/food/asian_
suffix : .png
id : 58daa1558bbb0b01f18ec1d3
name : Cha Chaan Teng
pluralName : Cha Chaan Tengs
shortName : Cha Chaan Teng
prefix : https://ss3.4sqi.net/img/categories_v2/food/asian_
su

shortName : Deli / Bodega
prefix : https://ss3.4sqi.net/img/categories_v2/food/deli_
suffix : .png
id : 4bf58dd8d48988d1d0941735
name : Dessert Shop
pluralName : Dessert Shops
shortName : Desserts
prefix : https://ss3.4sqi.net/img/categories_v2/food/dessert_
suffix : .png
id : 4bf58dd8d48988d1bc941735
name : Cupcake Shop
pluralName : Cupcake Shops
shortName : Cupcakes
prefix : https://ss3.4sqi.net/img/categories_v2/food/cupcakes_
suffix : .png
id : 512e7cae91d4cbb4e5efe0af
name : Frozen Yogurt Shop
pluralName : Frozen Yogurt Shops
shortName : Yogurt
prefix : https://ss3.4sqi.net/img/categories_v2/food/frozenyogurt_
suffix : .png
id : 5f2c407c5b4c177b9a6dc536
name : Gelato Shop
pluralName : Gelato Shops
shortName : Gelato
prefix : https://ss3.4sqi.net/img/categories_v2/food/icecream_
suffix : .png
id : 4bf58dd8d48988d1c9941735
name : Ice Cream Shop
pluralName : Ice Cream Shops
shortName : Ice Cream
prefix : https://ss3.4sqi.net/img/categories_v2/food/icecream_
suffix : .png
id : 5744ccd

suffix : .png
id : 54135bf5e4b08f3d2429dfe3
name : Dosa Place
pluralName : Dosa Places
shortName : Dosa
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 54135bf5e4b08f3d2429dfe8
name : Goan Restaurant
pluralName : Goan Restaurants
shortName : Goan
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 54135bf5e4b08f3d2429dfe9
name : Gujarati Restaurant
pluralName : Gujarati Restaurants
shortName : Gujarati
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 54135bf5e4b08f3d2429dfe6
name : Hyderabadi Restaurant
pluralName : Hyderabadi Restaurants
shortName : Hyderabadi
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 54135bf5e4b08f3d2429dfdf
name : Indian Chinese Restaurant
pluralName : Indian Chinese Restaurants
shortName : Indian Chinese
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 54135bf5e4b08f3d2429dfe4
name : Indian Sweet Shop
p

pluralName : Portuguese Restaurants
shortName : Portuguese
prefix : https://ss3.4sqi.net/img/categories_v2/food/portuguese_
suffix : .png
id : 56aa371be4b08b9a8d5734c7
name : Poutine Place
pluralName : Poutine Places
shortName : Poutine Place
prefix : https://ss3.4sqi.net/img/categories_v2/food/fastfood_
suffix : .png
id : 4bf58dd8d48988d1c4941735
name : Restaurant
pluralName : Restaurants
shortName : Restaurant
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 5293a7563cf9994f4e043a44
name : Russian Restaurant
pluralName : Russian Restaurants
shortName : Russian
prefix : https://ss3.4sqi.net/img/categories_v2/food/russian_
suffix : .png
id : 52e928d0bcbc57f1066b7e9d
name : Blini House
pluralName : Blini Houses
shortName : Blini
prefix : https://ss3.4sqi.net/img/categories_v2/food/default_
suffix : .png
id : 52e928d0bcbc57f1066b7e9c
name : Pelmeni House
pluralName : Pelmeni Houses
shortName : Pelmeni
prefix : https://ss3.4sqi.net/img/categories_v2/food/de

shortName : Cycle Studio
prefix : https://ss3.4sqi.net/img/categories_v2/shops/bikeshop_
suffix : .png
id : 4bf58dd8d48988d105941735
name : Gym Pool
pluralName : Gym Pools
shortName : Pool
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_
suffix : .png
id : 52f2ab2ebcbc57f1066b8b48
name : Gymnastics Gym
pluralName : Gymnastics Gyms
shortName : Gymnastics Gym
prefix : https://ss3.4sqi.net/img/categories_v2/building/gym_
suffix : .png
id : 4bf58dd8d48988d176941735
name : Gym
pluralName : Gyms
shortName : Gym
prefix : https://ss3.4sqi.net/img/categories_v2/building/gym_
suffix : .png
id : 4bf58dd8d48988d101941735
name : Martial Arts School
pluralName : Martial Arts Schools
shortName : Martial Arts
prefix : https://ss3.4sqi.net/img/categories_v2/shops/gym_martialarts_
suffix : .png
id : 58daa1558bbb0b01f18ec203
name : Outdoor Gym
pluralName : Outdoor Gyms
shortName : Outdoor Gym
prefix : https://ss3.4sqi.net/img/categories_v2/building/gym_
suffix : .png
id : 5744ccdfe4b0

pluralName : Counties
shortName : County
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_
suffix : .png
id : 530e33ccbcbc57f1066bbff7
name : Country
pluralName : Countries
shortName : Country
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_
suffix : .png
id : 4f2a25ac4b909258e854f55f
name : Neighborhood
pluralName : Neighborhoods
shortName : Neighborhood
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_
suffix : .png
id : 530e33ccbcbc57f1066bbff8
name : State
pluralName : States
shortName : State
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_
suffix : .png
id : 530e33ccbcbc57f1066bbff3
name : Town
pluralName : Towns
shortName : Town
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_
suffix : .png
id : 530e33ccbcbc57f1066bbff9
name : Village
pluralName : Villages
shortName : Village
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neigh

shortName : Hindu Temple
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_
suffix : .png
id : 5744ccdfe4b0c0459246b4ac
name : Kingdom Hall
pluralName : Kingdom Halls
shortName : Kingdom Hall
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_
suffix : .png
id : 52e81612bcbc57f1066b7a40
name : Monastery
pluralName : Monasteries
shortName : Monastery
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_
suffix : .png
id : 4bf58dd8d48988d138941735
name : Mosque
pluralName : Mosques
shortName : Mosque
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_mosque_
suffix : .png
id : 52e81612bcbc57f1066b7a41
name : Prayer Room
pluralName : Prayer Rooms
shortName : Prayer Room
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_
suffix : .png
id : 4eb1d80a4b900d56c88a45ff
name : Shrine
pluralName : Shrines
shortName : Shrine
prefix : https://ss3.4sqi.net/img/categories_v2/building/religious_shrine_
suffix : .png
id : 5

pluralName : Gaming Cafes
shortName : Gaming Cafe
prefix : https://ss3.4sqi.net/img/categories_v2/shops/gamingcafe_
suffix : .png
id : 4eb1c0253b7b52c0e1adc2e9
name : Garden Center
pluralName : Garden Centers
shortName : Garden Center
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/gardencenter_
suffix : .png
id : 4bf58dd8d48988d113951735
name : Gas Station
pluralName : Gas Stations
shortName : Gas Station
prefix : https://ss3.4sqi.net/img/categories_v2/shops/gas_
suffix : .png
id : 4bf58dd8d48988d128951735
name : Gift Shop
pluralName : Gift Shops
shortName : Gift Shop
prefix : https://ss3.4sqi.net/img/categories_v2/shops/giftshop_
suffix : .png
id : 52f2ab2ebcbc57f1066b8b19
name : Gun Shop
pluralName : Gun Shops
shortName : Gun Shop
prefix : https://ss3.4sqi.net/img/categories_v2/parks_outdoors/gun_
suffix : .png
id : 4bf58dd8d48988d112951735
name : Hardware Store
pluralName : Hardware Stores
shortName : Hardware
prefix : https://ss3.4sqi.net/img/categories_v2/shops/har

shortName : Duty-free
prefix : https://ss3.4sqi.net/img/categories_v2/shops/default_
suffix : .png
id : 4bf58dd8d48988d1f6931735
name : General Travel
pluralName : General Travel
shortName : Travel
prefix : https://ss3.4sqi.net/img/categories_v2/travel/default_
suffix : .png
id : 56aa371ce4b08b9a8d57356e
name : Heliport
pluralName : Heliports
shortName : Heliport
prefix : https://ss3.4sqi.net/img/categories_v2/travel/default_
suffix : .png
id : 4bf58dd8d48988d1fa931735
name : Hotel
pluralName : Hotels
shortName : Hotel
prefix : https://ss3.4sqi.net/img/categories_v2/travel/hotel_
suffix : .png
id : 4bf58dd8d48988d1f8931735
name : Bed & Breakfast
pluralName : Bed & Breakfasts
shortName : B & B
prefix : https://ss3.4sqi.net/img/categories_v2/travel/bedandbreakfast_
suffix : .png
id : 4f4530a74b9074f6e4fb0100
name : Boarding House
pluralName : Boarding Houses
shortName : Boarding
prefix : https://ss3.4sqi.net/img/categories_v2/travel/hotel_
suffix : .png
id : 4bf58dd8d48988d1ee931735
name

In [8]:
#find indexes to later reference in grouping categories
print(smallCatList.index('Arts & Entertainment'),
      smallCatList.index('College & University'),
      smallCatList.index('Event'),
      smallCatList.index('Food'),
      smallCatList.index('Nightlife Spot'),
      smallCatList.index('Outdoors & Recreation'),
      smallCatList.index('Professional & Other Places'),
      smallCatList.index('Residence'),
      smallCatList.index('Shop & Service'),
      smallCatList.index('Travel & Transport'))
      

0 67 106 119 477 504 616 728 734 913


In [9]:
#here are the new broad categories - notably, this is copy/pasted into the other notebook
arts = smallCatList[0:67]
uni = smallCatList[67:106]
event = smallCatList[106:119]
food = smallCatList[119:477]
nightlife = smallCatList[477:504]
rec = smallCatList[504:616]
pro = smallCatList[616:728]
reside = smallCatList[728:734]
shop = smallCatList[734:913]
travel = smallCatList[913:]


In [10]:
# this is a magic function to easily store a variable and later import it in another notebook
%store smallCatList

Stored 'smallCatList' (list)
